In [ ]:
# === Cell 1: Setup & Imports ===
# You can run this as-is in Colab. If using local Jupyter, set MOUNT_GOOGLE_DRIVE=False.

MOUNT_GOOGLE_DRIVE = True  # set to False if not using Google Drive

if MOUNT_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

import os, math, pathlib, sys, random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

print("TF version:", tf.__version__)

# Reproducibility
SEED = 13
random.seed(SEED)
tf.random.set_seed(SEED)

# === CHANGE: Set your data folder here ===
# Folder structure should be:
# DATA_DIR/
#   ├── class0/
#   └── class1/
DATA_DIR = "/content/drive/MyDrive/Images/Images"  # <-- update if needed

# Basic sanity checks
print("Exists?", os.path.isdir(DATA_DIR))
if os.path.isdir(DATA_DIR):
    print("Top-level items:", os.listdir(DATA_DIR)[:10])

# Hyperparams
IMG_SIZE = (224, 224)
BATCH    = 16
AUTOTUNE = tf.data.AUTOTUNE
VAL_SPLIT = 0.20

# Mixed precision (optional but safe on GPU)
try:
    mixed_precision.set_global_policy("mixed_float16")
    print("Mixed precision policy:", mixed_precision.global_policy())
except Exception as e:
    print("Mixed precision not set:", e)


Mounted at /content/drive
TF version: 2.19.0
Exists? True
Top-level items: ['No_Appendicitis_Images', 'Appendicitis_Images']
Mixed precision policy: <DTypePolicy "mixed_float16">


In [ ]:
# === Cell 2: Build Train/Val Datasets ===
# === CHANGE: Use label_mode='int' so we can filter by integer labels for oversampling
train_ds_raw = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="int",                # <--- CHANGE: 'int' not 'binary'
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="int",                # <--- keep labels as int
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH
)

# Keep class names for reference
class_names = train_ds_raw.class_names
print("Detected classes:", class_names)
num_classes = len(class_names)
assert num_classes == 2, "This notebook assumes binary classification."


Found 1721 files belonging to 2 classes.
Using 1377 files for training.
Found 1721 files belonging to 2 classes.
Using 344 files for validation.
Detected classes: ['Appendicitis_Images', 'No_Appendicitis_Images']


In [ ]:
# === Cell 3: Count Class Distribution in TRAIN (before oversampling) ===
# We'll iterate train_ds_raw *once* just to get counts and set steps_per_epoch cleanly.

def count_labels(dataset):
    counts = {0: 0, 1: 0}
    for batch_x, batch_y in dataset:
        # batch_y shape (B,)
        for y in batch_y.numpy().tolist():
            counts[int(y)] += 1
    return counts

train_counts = count_labels(train_ds_raw)
n0, n1 = train_counts[0], train_counts[1]
total_train = n0 + n1

print(f"Train counts -> {class_names[0]}: {n0}, {class_names[1]}: {n1}, total: {total_train}")

# Define steps per epoch (we'll keep an epoch roughly equivalent to one pass over original count)
steps_per_epoch = math.ceil(total_train / BATCH)
val_steps = None  # Keras will infer

print("steps_per_epoch:", steps_per_epoch)


Train counts -> Appendicitis_Images: 1074, No_Appendicitis_Images: 303, total: 1377
steps_per_epoch: 87


In [ ]:
# === Cell 4: Build Oversampled TRAIN Dataset ===
# === CHANGE: True oversampling via sample_from_datasets ===

# Unbatch to items (image, label)
train_unbatched = train_ds_raw.unbatch()

# Split by label
ds_class0 = train_unbatched.filter(lambda x, y: tf.equal(y, 0)).repeat()
ds_class1 = train_unbatched.filter(lambda x, y: tf.equal(y, 1)).repeat()

# Sample equally from both classes
oversampled_train = tf.data.Dataset.sample_from_datasets(
    [ds_class0, ds_class1],
    weights=[0.5, 0.5],
    seed=SEED
)

# Rebatch to BATCH and add performance options
train_ds = (
    oversampled_train
    .batch(BATCH, drop_remainder=False)
    .prefetch(AUTOTUNE)
)

# Validation pipeline: cached & prefetched (no oversampling)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

print("Oversampled training dataset is ready.")


Oversampled training dataset is ready.


In [ ]:
# === Cell 5: Augmentation & Preprocessing (Ultrasound-tuned) ===
from tensorflow.keras.applications import densenet
preprocess_input = densenet.preprocess_input

# NOTE:
# - Removed horizontal flip (laterality matters for appendix).
# - Keep small rotation/zoom/translation.
# - Add intensity/contrast jitter + slight Gaussian noise.

data_augmentation = keras.Sequential([
    layers.RandomRotation(0.06),         # small; ultrasound orientation shifts
    layers.RandomZoom(0.10),
    layers.RandomTranslation(0.05, 0.05),
    layers.RandomContrast(0.15),         # intensity/contrast variations
    layers.GaussianNoise(0.02),          # light speckle-like noise
], name="data_augmentation")


In [ ]:
# === REPLACE cell 6: Build Model (DenseNet201 + sturdier head) ===
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import densenet
from tensorflow import keras
from tensorflow.keras import layers

preprocess_input = densenet.preprocess_input

def build_model(img_size=IMG_SIZE, dropout_rate=0.5, l2_reg=1e-6):
    inputs = layers.Input(shape=img_size + (3,), name="input_image")
    x = data_augmentation(inputs)
    x = layers.Lambda(preprocess_input, name="preprocess")(x)

    base = DenseNet201(include_top=False, weights="imagenet", input_tensor=x)
    base.trainable = False

    x = layers.GlobalAveragePooling2D(name="gap")(base.output)
    x = layers.Dropout(dropout_rate, name="dropout")(x)
    outputs = layers.Dense(
        1, activation="sigmoid", dtype="float32",
        kernel_regularizer=keras.regularizers.l2(l2_reg),
        name="pred"
    )(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name="DenseNet201_binary")
    return model, base

model, base = build_model()
model.summary()


74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "DenseNet201_binary"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_augmentation   │ (None, 224, 224,  │          0 │ input_image[0][0] │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ preprocess (Lambda) │ (None, 224, 224,  │          0 │ data_augmentatio… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ preprocess[0][0]  │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con

 Total params: 18,323,905 (69.90 MB)

 Trainable params: 1,921 (7.50 KB)

 Non-trainable params: 18,321,984 (69.89 MB)

In [ ]:
# === Cell 7: Compile (Stage 1) & Callbacks (AdamW + label smoothing) ===
# === REPLACE: Compile Stage 1 (no class_weight) ===
initial_lr = 3e-4  # a bit higher to learn head faster

model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=initial_lr, weight_decay=1e-4),
    loss=keras.losses.BinaryCrossentropy(label_smoothing=0.02),
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.AUC(curve="PR", name="auprc"),
        tf.keras.metrics.Precision(name="prec"),
        tf.keras.metrics.Recall(name="rec"),
    ],
)

cbs_stage1 = [
    keras.callbacks.ModelCheckpoint("chkpt_stage1.keras", monitor="val_auc", mode="max", save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
]

EPOCHS_STAGE1 = 20
history1 = model.fit(
    train_ds,                          # <-- keep your oversampled train ds
    validation_data=val_ds,
    epochs=EPOCHS_STAGE1,
    steps_per_epoch=steps_per_epoch,
    callbacks=cbs_stage1,
    verbose=1,
)


Epoch 1/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.5173 - auc: 0.5179 - auprc: 0.4984 - loss: 0.7756 - prec: 0.5121 - rec: 0.5028
Epoch 1: val_auc improved from -inf to 0.54977, saving model to chkpt_stage1.keras
87/87 ━━━━━━━━━━━━━━━━━━━━ 108s 890ms/step - accuracy: 0.5174 - auc: 0.5180 - auprc: 0.4986 - loss: 0.7754 - prec: 0.5122 - rec: 0.5027 - val_accuracy: 0.6570 - val_auc: 0.5498 - val_auprc: 0.2836 - val_loss: 0.6395 - val_prec: 0.2651 - val_rec: 0.2785 - learning_rate: 3.0000e-04
Epoch 2/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.5341 - auc: 0.5397 - auprc: 0.5027 - loss: 0.7615 - prec: 0.5069 - rec: 0.5275
Epoch 2: val_auc improved from 0.54977 to 0.61521, saving model to chkpt_stage1.keras
87/87 ━━━━━━━━━━━━━━━━━━━━ 28s 321ms/step - accuracy: 0.5339 - auc: 0.5397 - auprc: 0.5027 - loss: 0.7615 - prec: 0.5068 - rec: 0.5269 - val_accuracy: 0.7064 - val_auc: 0.6152 - val_auprc: 0.3297 - val_loss: 0.5965 - val_prec: 0.3036 - val_rec: 0.2152 - learni

In [ ]:
# === Corrected Stage 2 fine-tune (Option A, no MixUp) ===
from tensorflow.keras.layers import BatchNormalization

base.trainable = True

# Unfreeze last ~220 layers but keep BN frozen
for i, layer in enumerate(base.layers):
    if i < len(base.layers) - 220:
        layer.trainable = False
    else:
        if isinstance(layer, BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True

finetune_lr = 1e-5  # slightly higher than 5e-6 to avoid underfitting

optimizer_ft = keras.optimizers.AdamW(
    learning_rate=finetune_lr,
    weight_decay=5e-5
)
optimizer_ft.clipnorm = 1.0  # gradient clipping

# BCE with small label smoothing since no MixUp
loss_ft = keras.losses.BinaryCrossentropy(label_smoothing=0.02)

model.compile(
    optimizer=optimizer_ft,
    loss=loss_ft,
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.AUC(curve="PR", name="auprc"),
        tf.keras.metrics.Precision(name="prec"),
        tf.keras.metrics.Recall(name="rec"),
    ],
)

cbs_stage2 = [
    keras.callbacks.ModelCheckpoint("chkpt_stage2.keras", monitor="val_auc", mode="max", save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=1),
    # keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=6, restore_best_weights=True, verbose=1),
]

EPOCHS_STAGE2 = 15
history2 = model.fit(
    train_ds,              # <-- ONLY train_ds here
    validation_data=val_ds,
    epochs=EPOCHS_STAGE2,
    steps_per_epoch=steps_per_epoch,
    callbacks=cbs_stage2,
    verbose=1,
)


Epoch 1/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.8419 - auc: 0.9175 - auprc: 0.8889 - loss: 0.3816 - prec: 0.8208 - rec: 0.8684
Epoch 1: val_auc improved from -inf to 0.73606, saving model to chkpt_stage2.keras
87/87 ━━━━━━━━━━━━━━━━━━━━ 73s 441ms/step - accuracy: 0.8420 - auc: 0.9176 - auprc: 0.8891 - loss: 0.3815 - prec: 0.8209 - rec: 0.8685 - val_accuracy: 0.7238 - val_auc: 0.7361 - val_auprc: 0.4151 - val_loss: 0.6362 - val_prec: 0.4298 - val_rec: 0.6203 - learning_rate: 1.0000e-05
Epoch 2/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step - accuracy: 0.8230 - auc: 0.9052 - auprc: 0.8742 - loss: 0.4053 - prec: 0.8014 - rec: 0.8323
Epoch 2: val_auc did not improve from 0.73606
87/87 ━━━━━━━━━━━━━━━━━━━━ 27s 311ms/step - accuracy: 0.8231 - auc: 0.9053 - auprc: 0.8743 - loss: 0.4051 - prec: 0.8016 - rec: 0.8324 - val_accuracy: 0.7064 - val_auc: 0.7261 - val_auprc: 0.3956 - val_loss: 0.6369 - val_prec: 0.4068 - val_rec: 0.6076 - learning_rate: 1.0000e-05
Epoch 3/15
87/87 ━━━━

In [ ]:
# === Cell 11: Evaluate on Validation ===
val_metrics = model.evaluate(val_ds, verbose=1)
print("Validation metrics [loss, acc, auc, auprc, prec, rec]:")
print(val_metrics)


22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.7537 - auc: 0.7452 - auprc: 0.4248 - loss: 0.6780 - prec: 0.4785 - rec: 0.4731
Validation metrics [loss, acc, auc, auprc, prec, rec]:
[0.6677762269973755, 0.7529069781303406, 0.7372103929519653, 0.41800540685653687, 0.4642857015132904, 0.49367088079452515]


In [1]:
# === Cell 12: Optional Confusion Matrix on Validation ===
# Converts labels to a flat list and runs predictions to compute confusion matrix.

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true = []
y_pred = []

for batch_x, batch_y in val_ds:
    preds = model.predict(batch_x, verbose=0).ravel()
    y_pred.extend((preds >= 0.5).astype(int))
    y_true.extend(batch_y.numpy().astype(int))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

cm = confusion_matrix(y_true, y_pred, labels=[0,1])
print("Confusion matrix:\n", cm)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


NameError: name 'model' is not defined

In [ ]:
# === Threshold Sweep on Validation ===
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# Get probabilities on validation
y_true = []
y_prob = []
for bx, by in val_ds:
    p = model.predict(bx, verbose=0).ravel()
    y_prob.extend(p.tolist())
    y_true.extend(by.numpy().astype(int).tolist())
y_true = np.array(y_true)
y_prob = np.array(y_prob)

best_thr, best_acc = 0.5, 0.0
ths = np.linspace(0.1, 0.9, 33)
for t in ths:
    yp = (y_prob >= t).astype(int)
    acc = accuracy_score(y_true, yp)
    if acc > best_acc:
        best_acc, best_thr = acc, t

prec, rec, f1, _ = precision_recall_fscore_support(y_true, (y_prob>=best_thr).astype(int), average='binary')
print(f"Best threshold on VAL: {best_thr:.3f} | Acc={best_acc:.4f}  Prec={prec:.3f}  Rec={rec:.3f}  F1={f1:.3f}")


Best threshold on VAL: 0.800 | Acc=0.7878  Prec=0.594  Rec=0.241  F1=0.342


In [ ]:
# === Test-Time Augmentation (TTA) helper ===
def tta_predict(model, batch, tta_times=5):
    # apply only light transforms consistent with training; avoid flips
    preds = []
    for _ in range(tta_times):
        x_aug = batch  # if using layers inside model, just run multiple passes
        p = model.predict(x_aug, verbose=0).ravel()
        preds.append(p)
    return np.mean(np.stack(preds, axis=0), axis=0)

# Example usage on validation (works because aug is inside the model):
y_true, y_prob_tta = [], []
for bx, by in val_ds:
    p = tta_predict(model, bx, tta_times=5)
    y_prob_tta.extend(p.tolist())
    y_true.extend(by.numpy().astype(int).tolist())
y_true = np.array(y_true); y_prob_tta = np.array(y_prob_tta)

# Use best_thr from the previous cell:
from sklearn.metrics import accuracy_score
acc_tta = accuracy_score(y_true, (y_prob_tta >= best_thr).astype(int))
print(f"TTA Validation Accuracy @thr={best_thr:.3f}: {acc_tta:.4f}")


TTA Validation Accuracy @thr=0.800: 0.7878
